In [3]:
import numpy as np
import pandas as pd
import sys
import os

In [45]:
# Info of original dataset
FilePath = os.path.join(os.getcwd(), 'Windows_Data/dataset_4f.csv')
dataset = pd.read_csv(FilePath)
print(dataset.describe())
fall_set = dataset.loc[dataset['label'] == 1]
normal_set = dataset.loc[dataset['label'] == 0]
print('Number of fall-samples: {}'.format(fall_set.shape[0]))
print('Number of normal-samples: {}'.format(normal_set.shape[0]))
print('Falls / Normal = {}'.format(fall_set.shape[0] / normal_set.shape[0]))

           mean_smv       max_smv       min_smv      duration         label
count  56917.000000  56917.000000  56917.000000  56917.000000  56917.000000
mean      10.249079     17.747558      5.445815     -7.426305      0.032855
std        0.778187      6.408102      3.562756    347.463460      0.178258
min        7.068501      9.547117      0.030987   -995.000000      0.000000
25%        9.881977     10.075159      2.505617   -205.000000      0.000000
50%        9.993102     19.697805      4.566719    -14.000000      0.000000
75%       10.543813     23.092690      9.627163    181.000000      0.000000
max       15.180757     32.885551     10.015043    996.000000      1.000000
Number of fall-samples: 1870
Number of normal-samples: 55047
Falls / Normal = 0.0339709702617763


In [46]:
dataset_no_dups = dataset.drop_duplicates(subset=None, keep=False)
print(dataset_no_dups.describe())
fall_set_np = dataset_no_dups.loc[dataset['label'] == 1]
normal_set_np = dataset_no_dups.loc[dataset['label'] == 0]
print('Number of fall-samples (without duplicates): {}'.format(fall_set_np.shape[0]))
print('Number of normal-samples (without duplicates): {}'.format(normal_set_np.shape[0]))
print('Falls / Normal = {} (without duplicates)'.format(fall_set_np.shape[0] / normal_set_np.shape[0]))

           mean_smv       max_smv       min_smv      duration         label
count  56917.000000  56917.000000  56917.000000  56917.000000  56917.000000
mean      10.249079     17.747558      5.445815     -7.426305      0.032855
std        0.778187      6.408102      3.562756    347.463460      0.178258
min        7.068501      9.547117      0.030987   -995.000000      0.000000
25%        9.881977     10.075159      2.505617   -205.000000      0.000000
50%        9.993102     19.697805      4.566719    -14.000000      0.000000
75%       10.543813     23.092690      9.627163    181.000000      0.000000
max       15.180757     32.885551     10.015043    996.000000      1.000000
Number of fall-samples (without duplicates): 1870
Number of normal-samples (without duplicates): 55047
Falls / Normal = 0.0339709702617763 (without duplicates)


In [47]:
fall_set_np.describe()

,mean_smv,max_smv,min_smv,duration,label
count,1870.000000,1870.000000,1870.000000,1870.000000,1870.0
mean,10.033300,25.652472,3.059850,14.412834,1.0
std,0.214833,2.796419,1.475822,72.491306,0.0
min,9.339711,16.886193,0.302170,-362.000000,1.0
25%,9.881906,23.719808,1.890238,-31.750000,1.0
50%,10.022019,25.769089,2.986808,17.000000,1.0
75%,10.178270,27.555061,4.046217,53.000000,1.0
max,10.828897,32.885551,7.907462,432.000000,1.0


In [48]:
normal_set_np.describe()

,mean_smv,max_smv,min_smv,duration,label
count,55047.000000,55047.000000,55047.000000,55047.000000,55047.0
mean,10.256409,17.479021,5.526868,-8.168202,0.0
std,0.789269,6.324424,3.584762,353.039795,0.0
min,7.068501,9.547117,0.030987,-995.000000,0.0
25%,9.881977,10.064263,2.553806,-217.000000,0.0
50%,9.986674,19.374507,4.704689,-15.000000,0.0
75%,10.561412,22.855407,9.650719,192.000000,0.0
max,15.180757,32.430852,10.015043,996.000000,0.0


In [49]:
TrnSet = dataset_no_dups.drop(dataset_no_dups[(dataset_no_dups.label == 0) & (dataset_no_dups.max_smv < 17)].index)
print(TrnSet.loc[dataset['label'] == 1].describe())
print(TrnSet.loc[dataset['label'] == 0].describe())
X = TrnSet[['mean_smv','max_smv', 'min_smv', 'duration']].values
y = TrnSet[['label']].values

          mean_smv      max_smv      min_smv     duration   label
count  1870.000000  1870.000000  1870.000000  1870.000000  1870.0
mean     10.033300    25.652472     3.059850    14.412834     1.0
std       0.214833     2.796419     1.475822    72.491306     0.0
min       9.339711    16.886193     0.302170  -362.000000     1.0
25%       9.881906    23.719808     1.890238   -31.750000     1.0
50%      10.022019    25.769089     2.986808    17.000000     1.0
75%      10.178270    27.555061     4.046217    53.000000     1.0
max      10.828897    32.885551     7.907462   432.000000     1.0
           mean_smv       max_smv       min_smv      duration    label
count  32151.000000  32151.000000  32151.000000  32151.000000  32151.0
mean      10.537224     22.464480      2.777710    -10.489409      0.0
std        0.931833      2.728021      1.783888    378.240231      0.0
min        7.068501     17.000608      0.030987   -993.000000      0.0
25%       10.155082     20.439339      0.993152   -

In [50]:
# 5-fold cross validation (GridsearchCV) & apply undersampling and oversampling on each fold
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [51]:
def imb_pipeline(clf_name, clf, X, y, params):
    
    over = SMOTE(sampling_strategy=0.1)
    under = RandomUnderSampler(sampling_strategy=0.5)
    
    if clf_name == 'LinearSVC':
        normalize = StandardScaler()
        steps = [
        ('normalization', normalize),
        ('oversampling', over),
        ('undersampling', under),
        ('classification', clf)]
    else:
        steps = [
        ('oversampling', over),
        ('undersampling', under),
        ('classification', clf)]
    
    model = Pipeline(steps=steps)
    
    score={'AUC':'roc_auc', 
           'RECALL':'recall',
           'PRECISION':'precision',
           'F1':'f1'}
    
    gcv = GridSearchCV(estimator=model, param_grid=params, cv=5, scoring=score, n_jobs=12, refit='F1',
                       return_train_score=True)
    gcv.fit(X, y)

    return gcv



In [52]:
setting_LogisticRegression = {'classification__penalty': ('l1', 'l2'), 
                              'classification__C': (0.001, 0.01, 0.1, 1.0, 10, 100)}

setting_SVC = {'classification__C': [0.01, 0.1, 1, 10, 100], 
               'classification__gamma': [1e-1, 1e-2, 1e-3, 1e-4]}

setting_RandomForest = {'classification__n_estimators': (100, 100, 200, 300, 400, 500), 
                        'classification__max_depth': (10, 20, 30, 40, 50, 60, 70, 80, None),
                        'classification__min_samples_leaf':[1, 2, 4],
                        'classification__min_samples_split':[2, 5, 10]}

params = [setting_LogisticRegression, setting_SVC, setting_RandomForest]

In [53]:
classifiers = [
 ('Logistic Regression', LogisticRegression(random_state=0)),
 ('SVC', SVC(random_state=0)),
 ('Random Forst',RandomForestClassifier(bootstrap=True,random_state=0))]


In [54]:
for param, classifier in zip(params, classifiers):
    print("Working on {}...".format(classifier[0]))
    clf = imb_pipeline(classifier[0], classifier[1], X, y, param) 
    print("Best parameter for {} is {}".format(classifier[0], clf.best_params_))
    print("Best `F1` for {} is {}".format(classifier[0], clf.best_score_))
    print('-'*100)
    print('\n')

Working on Logistic Regression...


C:\ProgramData\Anaconda3\envs\oli\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.82630831        nan 0.8376595         nan 0.83501481
        nan 0.83366783        nan 0.83525786        nan 0.83889466]
  warnings.warn(
C:\ProgramData\Anaconda3\envs\oli\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the train scores are non-finite: [       nan 0.86440686        nan 0.88343481        nan 0.88570025
        nan 0.88709978        nan 0.88644487        nan 0.88680562]
  warnings.warn(
C:\ProgramData\Anaconda3\envs\oli\lib\site-packages\sklearn\model_selection\_search.py:918: UserWarning: One or more of the test scores are non-finite: [       nan 0.57219251        nan 0.6315508         nan 0.64010695
        nan 0.64010695        nan 0.64278075        nan 0.64919786]
  warnings.warn(
C:\ProgramData\Anaconda3\envs\oli\lib\site-packages\sklearn\model_selection\_search.py:

Best parameter for Logistic Regression is {'classification__C': 100, 'classification__penalty': 'l2'}
Best `F1` for Logistic Regression is 0.3645050196644971
----------------------------------------------------------------------------------------------------


Working on SVC...
Best parameter for SVC is {'classification__C': 100, 'classification__gamma': 0.0001}
Best `F1` for SVC is 0.5770286974233636
----------------------------------------------------------------------------------------------------


Working on Random Forst...
Best parameter for Random Forst is {'classification__max_depth': None, 'classification__min_samples_leaf': 2, 'classification__min_samples_split': 10, 'classification__n_estimators': 500}
Best `F1` for Random Forst is 0.7425155417027427
----------------------------------------------------------------------------------------------------


